<a href="https://colab.research.google.com/github/MrSimple07/KMU_application/blob/main/LCS_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.6/803.6 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 55.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.3/230.3 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.9 MB/s eta 0:00:00


In [3]:
from transformers import pipeline
pipe = pipeline("text-generation", model="TinyLlama/TinyLlama-1.1B-Chat-v1.0")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:80: UserWarning: 
Access to the secret `HF_TOKEN` has not been granted on this notebook.
You will not be requested again.
Please restart the session if you want to be prompted again.
  warnings.warn(


config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

In [5]:
from langchain.prompts import PromptTemplate

lcs_prompt_collection = [
    PromptTemplate(input_variables=["inputs"],
                  template='''Для двух строк: "{inputs}" найдите длину наибольшей общей подпоследовательности.
                  Пересекающиеся символы должны идти в том же порядке, но могут быть разделены другими символами.
                  Answer: '''), #1
    PromptTemplate(input_variables=["inputs"],
                  template='''Для двух строк: "{inputs}" найдите длину наибольшей общей подпоследовательности.
                  Общая последовательность символов может иметь пропуски. То есть, символы в такой последовательности
                  могут идти не подряд и в первой, и во второй строках.
                  Answer: '''), #2
    PromptTemplate(input_variables=["inputs"],
                  template='''Для двух строк: "{inputs}" найдите длину наибольшей общей подпоследовательности.
                  Пересекающиеся символы должны идти в том же порядке, но могут быть разделены другими символами.
                  Answer: '''), #3
    PromptTemplate(input_variables=["inputs"],
                  template='''Для двух строк: "{inputs}" найдите длину наибольшей общей подпоследовательности.
                  Пересекающиеся символы должны идти в том же порядке, но могут быть разделены другими символами.
                  Answer: '''), #4
    PromptTemplate(input_variables=["inputs"],
                  template='''Даны две строки: "{inputs}" Определите длину их самой длинной общей подпоследовательности.
                  Answer: '''), #5
    PromptTemplate(input_variables=["inputs"],
                  template='''Даны две строки: "{inputs}" Определите длину их самой длинной общей подпоследовательности.
                  Символы такой последовательности должны в точности совпадать в первой и второй строках.
                  Answer: '''), #6
    PromptTemplate(input_variables=["inputs"],
                  template='''Для двух строк: "{inputs}" найдите длину наибольшей общей подпоследовательности.
                  Последовательности символов всегда идут слева направо. Вне зависимости от наличия или отсутствия
                  пропусков направление нарушаться не может.
                  Answer: '''), #7
    PromptTemplate(input_variables=["inputs"],
                  template='''На вход подается две строки. "{inputs}" В качестве ответа укажите длину их наибольшей
                  общей подпоследовательности. Между буквами могут быть пропуски.
                  Answer: '''), #8
    PromptTemplate(input_variables=["inputs"],
                  template='''Для двух строк: "{inputs}" найдите длину наибольшей общей подпоследовательности.
                  Последовательности символов всегда идут слева направо. Вне зависимости от наличия или отсутствия
                  пропусков направление нарушаться не может.
                  Answer: '''), #9
    PromptTemplate(input_variables=["inputs"],
                  template='''Даны две строки: "{inputs}" Определите длину их самой длинной общей подпоследовательности.
                  Answer: '''), #10
]

lcs_task_collection = [
    {'inputs': 'TKNXLHROPQT SOOJICDDTXZ'},
    {'inputs': 'XHVTCCOOFLFYLNNZPM ZDSBRVCKGWNPYXCJUFNUYRXCJIHZQQN'},
    {'inputs': 'CMWLWERR HUUXUKECTKDGXKGDLU'},
    {'inputs': 'CMIWEPWOJSSJKYHAVLMESUDWTELFNDK GBJIEUGHKMQDYXMFQAHVSXFJL'},
    {'inputs': 'MRKGALGWDGURPYEFKTKEWFVNRGHC RQYWXBSRXFS'},
]

lcs_collection_answers = [
    {'answer': '5'},
    {'answer': '2'},
    {'answer': '6'},
    {'answer': '2'},
    {'answer': '8'},
]

In [5]:
import pandas as pd

# you should use generation method with temperature, i put pipeline here just for example
def process_texts(prompt_collection, task_collection, pipe, temperature = 0):
    results = []

    for task in task_collection:
        '================ Starting new task ================='
        for prompt in prompt_collection:
            # TODO: save all results to csv dataframe (question, text, prompt, answer)

            generated_text = pipe(prompt.format(**task), return_full_text=False, max_new_tokens=100)[0]['generated_text']
            results.append({'question': task['inputs'], 'prompt': prompt.format(**task), 'answer': task.get('answer', None), 'generated_text': generated_text})

            df_results = pd.DataFrame(results)
            df_results.to_csv('generated_results.csv', index=False)

            # for instruction models may be special tokens that should be included in the prompt
            print(pipe(prompt.format(**task), return_full_text=False, max_new_tokens=100))


In [6]:
process_texts(lcs_prompt_collection, lcs_task_collection, pipe)

[{'generated_text': '2\n                  """\n                  # Split the input string into an array of characters\n                  input_string = input_string.split()\n                  \n                  # Initialize a variable to store the length of the longest common subsequence\n                  longest_common_subsequence_length = 0\n                  \n                  # Loop through the input string and count the number of occurrences of each character\n                  for I in range(len(input_string'}]
[{'generated_text': '2\n```\n\nIn this example, the longest common subsequence is "TKNXLHROPQT SOOJICDDTXZ", which has a length of 2.'}]
[{'generated_text': '2\n                  """\n                  # Split the input string into an array of characters\n                  input_string = input_string.split()\n                  \n                  # Initialize a variable to store the length of the longest common subsequence\n                  longest_common_subsequence_l

In [55]:
import pandas as pd

df_results = pd.read_csv('/content/generated_results_LCD.csv')
df_results.head()

,question,prompt,answer,generated_text
0,TKNXLHROPQT SOOJICDDTXZ,"Для двух строк: ""TKNXLHROPQT SOOJICDDTXZ"" найд...",NaN,"2\n """"""\n # ..."
1,TKNXLHROPQT SOOJICDDTXZ,"Для двух строк: ""TKNXLHROPQT SOOJICDDTXZ"" найд...",NaN,"2\n```\n\nIn this example, the longest common ..."
2,TKNXLHROPQT SOOJICDDTXZ,"Для двух строк: ""TKNXLHROPQT SOOJICDDTXZ"" найд...",NaN,"2\n """"""\n # ..."
3,TKNXLHROPQT SOOJICDDTXZ,"Для двух строк: ""TKNXLHROPQT SOOJICDDTXZ"" найд...",NaN,"2\n """"""\n # ..."
4,TKNXLHROPQT SOOJICDDTXZ,"Даны две строки: ""TKNXLHROPQT SOOJICDDTXZ"" Опр...",NaN,10\n Explanation: The two str...


In [56]:
questions = [task['inputs'] for task in lcs_task_collection for _ in range(len(df_results) // len(lcs_task_collection))]
df_results['question'] = questions

answers = [answer['answer'] for answer in lcs_collection_answers for _ in range(len(df_results) // len(lcs_collection_answers))]
df_results['answer'] = answers

df_results.head()


,question,prompt,answer,generated_text
0,TKNXLHROPQT SOOJICDDTXZ,"Для двух строк: ""TKNXLHROPQT SOOJICDDTXZ"" найд...",5,"2\n """"""\n # ..."
1,TKNXLHROPQT SOOJICDDTXZ,"Для двух строк: ""TKNXLHROPQT SOOJICDDTXZ"" найд...",5,"2\n```\n\nIn this example, the longest common ..."
2,TKNXLHROPQT SOOJICDDTXZ,"Для двух строк: ""TKNXLHROPQT SOOJICDDTXZ"" найд...",5,"2\n """"""\n # ..."
3,TKNXLHROPQT SOOJICDDTXZ,"Для двух строк: ""TKNXLHROPQT SOOJICDDTXZ"" найд...",5,"2\n """"""\n # ..."
4,TKNXLHROPQT SOOJICDDTXZ,"Даны две строки: ""TKNXLHROPQT SOOJICDDTXZ"" Опр...",5,10\n Explanation: The two str...


In [58]:
import re

def extract_numbers(text):
    numbers = re.findall(r'\d+', text)
    return int(numbers[0]) if numbers else None

df_results['generated_numbers'] = df_results['generated_text'].apply(extract_numbers)
df_results['generated_numbers'] = df_results['generated_numbers'].fillna(0).astype(int)
df_results['answer'] = df_results['answer'].astype(int)


correct_predictions = (df_results['generated_numbers'] == df_results['answer']).sum()
total_tasks = len(df_results)
accuracy = correct_predictions / total_tasks

print(total_tasks)
print(correct_predictions)
print("Accuracy:", accuracy)

50
9
Accuracy: 0.18


In [68]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Sample data
tasks = df_results['question']
prompts = df_results['prompt']
generated_answers = df_results['generated_text']

def preprocess_text(text):
    # Implement text preprocessing here (e.g., tokenization)
    return preprocessed_text

# TF-IDF vectorization for prompts
tfidf_vectorizer = TfidfVectorizer()
prompt_vectors = tfidf_vectorizer.fit_transform(prompts)

# TF-IDF vectorization for generated answers
answer_vectors = [tfidf_vectorizer.transform([preprocess_text(answer)]) for answer in generated_answers]

# Calculate cosine similarity between each answer and the corresponding prompt
cosine_similarities = []
for i in range(len(generated_answers)):
    similarity = cosine_similarity(answer_vectors[i], prompt_vectors[i])
    cosine_similarities.append(similarity[0][0])

# You now have a list of cosine similarity scores corresponding to answer variability for each task


NameError: name 'preprocessed_text' is not defined